In [4]:
import pandas as pd

def split_and_find_matches(df):
    df['Tags'] = df['Tags'].apply(lambda x: ' '.join(str(x).split()) if pd.notnull(x) else '')
    train_df = df.sample(frac=0.6, random_state=1) 
    test_df = df.drop(train_df.index)
    
    def count_matching_tags(tags1, tags2):
        set1 = set(tags1.split())
        set2 = set(tags2.split())
        return len(set1.intersection(set2))
    
    
    for index, test_row in test_df.iterrows():
        train_df['MatchingTags'] = train_df['Tags'].apply(lambda x: count_matching_tags(x, test_row['Tags']))
        sorted_train_df = train_df.sort_values('MatchingTags', ascending=False)
        top_matches = sorted_train_df.head(1)
        
        if not top_matches.empty:
            top_row = top_matches.iloc[0]
            prompt_fewshot = (f"This is a code block, written in Java, that has a slow algorithmic time complexity:\n{test_row['GROUND']} "
                      f"\n\nI want you to optimize the code above such that the new code is more time efficient and serves the same objective as the code block above. I don't want you to give me any explanation, just return the newly generated code."
                      f"\nUse a similar optimization example as a reference. \nExample - The code: {top_row['GROUND']} "
                      f"\n\nis optimized to: {top_row['BEST']}"
                      f"\n\nMake sure it still passes the test case:"
                      f"\nInput:\n{test_row['input']}"
                      f"\nOutput:\n{test_row['output']}\n")
            prompt_oneshot = (f"This is a code block, written in Java, that has a slow algorithmic time complexity:\n{test_row['GROUND']} "
                      f"\n\nI want you to optimize the code above such that the new code is more time efficient and serves the same objective as the code block above. I don't want you to give me any explanation, just return the newly generated code."
                      f"\n\nMake sure it still passes the test case:"
                      f"\nInput:\n{test_row['input']}"
                      f"\nOutput:\n{test_row['output']}\n")
            
            test_df.loc[index, 'One Shot Prompt'] = prompt_oneshot
            test_df.loc[index, 'Few Shot Prompt'] = prompt_fewshot
        else:
            prompt = (f"This is a code block, written in Java, that has a slow algorithmic time complexity:\n{test_row['GROUND']} "
                      f"\n\nI want you to optimize the code above such that the new code is more time efficient and serves the same objective as the code block above. I don't want you to give me any explanation, just return the newly generated code."
                      f"\n\nMake sure it still passes the test case:"
                      f"\nInput:\n{test_row['input']}"
                      f"\nOutput:\n{test_row['output']}\n")
            test_df.loc[index, 'One Shot Prompt'] = prompt
            test_df.loc[index, 'Few Shot Prompt'] = ""

        test_df.loc[index, 'One Shot Response: GPT-4'] = ""
        test_df.loc[index, 'One Shot Response: GitHub Pilot'] = ""

        test_df.loc[index, 'Few Shot Response: GPT-4'] = ""
        test_df.loc[index, 'Few Shot Response: GitHub Pilot'] = ""
    
    return test_df


df = pd.read_csv('final_merged_with_tags.csv')
test_df_with_prompts = split_and_find_matches(df)


output_csv_path = 'final_merged_with_prompts_test_case.csv'
test_df_with_prompts.to_csv(output_csv_path, index=False)

SyntaxError: EOL while scanning string literal (982245284.py, line 26)